# A Simple Feed Forward Neural Network from Scratch

(C) 2023-2024 by [Damir Cavar](http://damir.cavar.me/)


**Prerequisites:**

In [ ]:
!pip install -U numpy
!pip install -U nltk
!pip install -U scipy

## Introduction

In [1]:
import numpy as np
from scipy.special import softmax

In [8]:
x = np.array([2, 3, 4, 5, 6, 7])
num_features = 5
W = np.random.rand(len(x), num_features)
b = np.random.rand(num_features)

In [9]:
x_reshaped = x.reshape(len(x), 1)
print(x_reshaped.T)
print(x_reshaped)


[[2 3 4 5 6 7]]
[[2]
 [3]
 [4]
 [5]
 [6]
 [7]]


In [10]:
W

array([[0.50500969, 0.61216835, 0.97560639, 0.40014872, 0.52546529],
       [0.75370475, 0.29997896, 0.04612736, 0.55810029, 0.09418851],
       [0.45652967, 0.73751977, 0.6362012 , 0.71395926, 0.6058033 ],
       [0.09339971, 0.75861677, 0.42965112, 0.63466228, 0.61543066],
       [0.70792968, 0.86335131, 0.344358  , 0.5734288 , 0.29693733],
       [0.52017696, 0.05920029, 0.60644483, 0.27791372, 0.98732493]])

In [11]:
b

array([0.60459173, 0.05570785, 0.58536412, 0.84748354, 0.70080312])

In [12]:
z = x_reshaped.T @ W + b

In [13]:
print(z)

[[14.05765941 14.51765422 13.67928118 14.73719912 16.22756423]]


In [14]:
def relu(x):
    return np.maximum(0,x)

In [15]:
a = relu(z)
a

array([[14.05765941, 14.51765422, 13.67928118, 14.73719912, 16.22756423]])

In [16]:
U = np.random.rand(len(a[0]), 1)

In [17]:
U

array([[0.21410325],
       [0.93751543],
       [0.16085734],
       [0.1212611 ],
       [0.92247533]])

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [5]:
print(softmax(np.array([[3.0, 1.0, 0.2]])))
print(sum(softmax(np.array([[30.0, 10.0, 2.0]]))[0]))

[[0.8360188  0.11314284 0.05083836]]
0.9999999999999999


In [ ]:
z2 = a @ U
print(z2)
sigmoid(z2[0][0])

## Sentiment Analysis Example

In [ ]:
import os
import csv

In [ ]:
experiment_data = []
with open(os.path.join('.', 'data', 'reviews.csv'), newline='') as csvfile:
    datareader = csv.reader(csvfile, delimiter=',', quotechar='"')
    header = next(datareader)
    for row in datareader:
        if len(row) == 2:
            experiment_data.append( [row[0].strip(), int(row[1].strip())] )

In [ ]:
print(experiment_data[:2])
print("Text:\n", experiment_data[0][0])
print("Value:\n", experiment_data[0][1])


In [ ]:
sentiment_dictionary = {}
with open(os.path.join('.', 'data', 'vader_lexicon.txt'), mode='r', encoding='utf-8') as ifile:
    lines = ifile.readlines()
    sentiment_dictionary = { y[0]: y[1] for y in [ x.split('\t') for x in lines ] if len(y) == 4 }

In [ ]:
print(sentiment_dictionary["adventurer"])

In [ ]:
from nltk.tokenize import word_tokenize
from collections import Counter
import math
import numpy as np

In [ ]:
pronouns = {"i", "me", "my", "mine", "you", "yours", "yourself", "myself", "we", "us", "our", "ours", "ourselves"}

In [ ]:
def get_me_the_vector(text):
    tokens = word_tokenize(text.lower())
    scores = [ float(sentiment_dictionary.get(t, 0.0)) for t in tokens ]
    positive = len([ s for s in scores if s > 0 ])
    negative = len([ s for s in scores if s < 0 ])
    if "no" in tokens:
        no_present = 1
    else:
        no_present = 0
    counts = Counter(tokens)
    pronoun_count = 0
    for x in set(counts.keys()).intersection(pronouns):
        pronoun_count += counts[x]
    if "!" in tokens:
        exclamation = 1
    else:
        exclamation = 0
    return (positive, negative, no_present, pronoun_count, exclamation, math.log(len(tokens)))

In [ ]:
vectors = [ get_me_the_vector(e[0]) for e in experiment_data ]

In [ ]:
print(vectors[1], "\n", experiment_data[1][0])

In [ ]:
def relu(x):
    return np.maximum(0, x)

In [ ]:
# x = np.array([2, 3, 4, 5, 6, 7])
num_features = 6
num_rows = 3
W = np.random.rand(num_rows, num_features)
print("W:\n", W)
b = np.random.rand(num_rows).reshape(1, num_rows).T
print("b:\n", b)

In [ ]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

In [ ]:
# we get back a 3-dimensional vector
U = np.random.rand(1, num_rows)
print("U:\n", U)
bu = np.random.rand(1, num_rows)
print("bu:\n", bu)

In [ ]:
results = []
for x, truth in zip(vectors, [ v[1] for v in experiment_data ]):
    x = np.array(x).reshape(1, len(x)).T
    print("x:\n", x)
    print("W:\n", W)
    print("b:\n", b)
    z = np.dot(W, x) + b
    print("z:\n", z)
    a = relu(z)
    print("a:\n", a)
    c = sigmoid(np.dot(U, a))
    print("c:\n", c)
    results.append( (truth, c) )

In [ ]:
for x in results[:10]:
    print(x)

In [ ]:
a = [1, 2, 3]
b = ["a", "b", "c"]
list(zip(a, b))